Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChains
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchains);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations        = 1:1000
Thinning interval = 1
Chains            = Chain1, Chain2, Chain3, Chain4
Samples per chain = 1000
parameters        = alpha, beta, sigma

Empirical Posterior Estimates:
parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 154.5948 0.2733   0.0043 0.0041 1000
 beta   0.9056 0.0428   0.0007 0.0007 1000
sigma   5.1069 0.1905   0.0030 0.0032 1000

Quantiles:
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
alpha 153.6480 154.4110 154.5990 154.7762 155.6110
 beta   0.7521   0.8775   0.9062   0.9340   1.0580
sigma   4.5054   4.9746   5.1038   5.2324   5.9968



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64,Float64,NamedTuple{(:internals, :parameters),Tuple{Array{Symbol,1},Array{Symbol,1}}},NamedTuple{(),Tuple{}}}"

Iterations        = 1:1000
Thinning interval = 1
Chains            = Chain1, Chain2, Chain3, Chain4
Samples per chain = 1000
internals         = 
parameters        = alpha, beta, sigma



Should be identical to earlier result

In [9]:
describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = Chain1, Chain2, Chain3, Chain4
Samples per chain = 1000
parameters        = alpha, beta, sigma

Empirical Posterior Estimates:
parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 154.5948 0.2733   0.0043 0.0041 1000
 beta   0.9056 0.0428   0.0007 0.0007 1000
sigma   5.1069 0.1905   0.0030 0.0032 1000

Quantiles:
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
alpha 153.6480 154.4110 154.5990 154.7762 155.6110
 beta   0.7521   0.8775   0.9062   0.9340   1.0580
sigma   4.5054   4.9746   5.1038   5.2324   5.9968



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*